In [1]:
# Dependencies
from bs4 import BeautifulSoup
import requests
from splinter import Browser
import pandas as pd

# Connection and Scraping

In [2]:
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

In [3]:
news_url = "https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest"
browser.visit(news_url)

In [4]:
html = browser.html
soup = BeautifulSoup(html, 'html.parser')
# print(soup.prettify())

In [5]:
news_title = soup.find("li", class_="slide").find("div", class_="content_title").text
print(news_title)
news_p = soup.find("li", class_="slide").find("div", class_="article_teaser_body").text
print(news_p)

After a Reset, Curiosity Is Operating Normally
Curiosity has returned to science operations and is once again exploring the clay unit. 


# Image url scraping

In [6]:
nasa_url = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
browser.visit(nasa_url)

In [7]:
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [8]:
featured_img_base = "https://www.jpl.nasa.gov"
featured_img_url_raw = soup.find("div", class_="carousel_items").find("article")["style"]
featured_img_url = featured_img_url_raw.split("'")[1]
featured_img_url = featured_img_base + featured_img_url
featured_img_url

'https://www.jpl.nasa.gov/spaceimages/images/wallpaper/PIA14316-1920x1200.jpg'

# Mars tweet scraping

In [9]:
weather_twitter_url = "https://twitter.com/marswxreport?lang=en"
browser.visit(weather_twitter_url)

In [10]:
html = browser.html
soup = BeautifulSoup(html, 'html.parser')
print(soup.prettify())
soup.save_as('news.html')

<!DOCTYPE html>
<html data-scribe-reduced-action-queue="true" lang="en" xmlns="http://www.w3.org/1999/xhtml">
 <head>
  <meta charset="utf-8"/>
  <script async="" src="//www.google-analytics.com/analytics.js">
  </script>
  <script nonce="">
   !function(){window.initErrorstack||(window.initErrorstack=[]),window.onerror=function(r,i,n,o,t){r.indexOf("Script error.")&gt;-1||window.initErrorstack.push({errorMsg:r,url:i,lineNumber:n,column:o,errorObj:t})}}();
  </script>
  <script id="bouncer_terminate_iframe" nonce="">
   if (window.top != window) {
  window.top.postMessage({'bouncer': true, 'event': 'complete'}, '*');
}
  </script>
  <script id="ttft_boot_data" nonce="">
   window.ttftData={"transaction_id":"00f9ab5f00a3bdc9.4ca9d0445d79e7bd\u003c:0027e149000cdd28","server_request_start_time":1551560918157,"user_id":null,"is_ssl":true,"rendered_on_server":true,"is_tfe":true,"client":"macaw-swift","tfe_version":"tsa_a\/1.0.1\/20190206.2140.12b8a68","ttft_browser":"chrome"};!function(){fu

TypeError: 'NoneType' object is not callable

In [13]:

tweets = soup.find("div", class_="stream").find("ol").find_all("li", class_="js-stream-item")
for tweet in tweets:
    tweet_text = tweet.find("div", class_="js-tweet-text-container").text
    if "Weather " in tweet_text:
        mars_weather = tweet_text.strip()
        break

mars_weather

'InSight sol 81 (2019-02-17), high -17/2F, low -95/-138F, pressure at 7.23hPa, winds from the WNW at 12 mph gusting to 37.8 mph\n\nWelcome to the Mars Weather team @NASAInSight!\nhttps://mars.nasa.gov/insight/weather/\xa0…pic.twitter.com/SH12FvcMfv'

# Fact scraping

In [14]:
facts_url = "https://space-facts.com/mars/"
tables = pd.read_html(facts_url)

In [15]:
facts_df = tables[0]
facts_df.columns = ["Fact","Value"]

# get rid of trailing colon
facts_df["Fact"] = facts_df["Fact"].str[:-1]
facts_df = facts_df.set_index("Fact")
facts_df

,Value
Fact,
Equatorial Diameter,"6,792 km"
Polar Diameter,"6,752 km"
Mass,6.42 x 10^23 kg (10.7% Earth)
Moons,2 (Phobos & Deimos)
Orbit Distance,"227,943,824 km (1.52 AU)"
Orbit Period,687 days (1.9 years)
Surface Temperature,-153 to 20 °C
First Record,2nd millennium BC
Recorded By,Egyptian astronomers


In [17]:
facts_table = facts_df.to_html()
facts_table = facts_table.replace('\n', '')
facts_table

'<table border="1" class="dataframe">  <thead>    <tr style="text-align: right;">      <th></th>      <th>Value</th>    </tr>    <tr>      <th>Fact</th>      <th></th>    </tr>  </thead>  <tbody>    <tr>      <th>Equatorial Diameter</th>      <td>6,792 km</td>    </tr>    <tr>      <th>Polar Diameter</th>      <td>6,752 km</td>    </tr>    <tr>      <th>Mass</th>      <td>6.42 x 10^23 kg (10.7% Earth)</td>    </tr>    <tr>      <th>Moons</th>      <td>2 (Phobos &amp; Deimos)</td>    </tr>    <tr>      <th>Orbit Distance</th>      <td>227,943,824 km (1.52 AU)</td>    </tr>    <tr>      <th>Orbit Period</th>      <td>687 days (1.9 years)</td>    </tr>    <tr>      <th>Surface Temperature</th>      <td>-153 to 20 °C</td>    </tr>    <tr>      <th>First Record</th>      <td>2nd millennium BC</td>    </tr>    <tr>      <th>Recorded By</th>      <td>Egyptian astronomers</td>    </tr>  </tbody></table>'

# Mars photo scraping

In [24]:
base_hemisphere_url = "https://astrogeology.usgs.gov"
hemisphere_url = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
browser.visit(hemisphere_url)

In [25]:
html = browser.html
soup = BeautifulSoup(html, 'html.parser')
    
hemisphere_image_urls = []

links = soup.find_all("div", class_="item")

for link in links:
    img_dict = {}
    title = link.find("h3").text
    next_link = link.find("div", class_="description").a["href"]
    full_next_link = base_hemisphere_url + next_link
    
    browser.visit(full_next_link)
    
    pic_html = browser.html
    pic_soup = BeautifulSoup(pic_html, 'html.parser')
    
    url = pic_soup.find("img", class_="wide-image")["src"]

    img_dict["title"] = title
    img_dict["img_url"] = base_hemisphere_url + url
    print(img_dict["img_url"])
    
    hemisphere_image_urls.append(img_dict)

https://astrogeology.usgs.gov/cache/images/cfa62af2557222a02478f1fcd781d445_cerberus_enhanced.tif_full.jpg
https://astrogeology.usgs.gov/cache/images/3cdd1cbf5e0813bba925c9030d13b62e_schiaparelli_enhanced.tif_full.jpg
https://astrogeology.usgs.gov/cache/images/ae209b4e408bb6c3e67b6af38168cf28_syrtis_major_enhanced.tif_full.jpg
https://astrogeology.usgs.gov/cache/images/7cf2da4bf549ed01c17f206327be4db7_valles_marineris_enhanced.tif_full.jpg


In [27]:
hemisphere_image_urls

[{'title': 'Cerberus Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/cfa62af2557222a02478f1fcd781d445_cerberus_enhanced.tif_full.jpg'},
 {'title': 'Schiaparelli Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/3cdd1cbf5e0813bba925c9030d13b62e_schiaparelli_enhanced.tif_full.jpg'},
 {'title': 'Syrtis Major Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/ae209b4e408bb6c3e67b6af38168cf28_syrtis_major_enhanced.tif_full.jpg'},
 {'title': 'Valles Marineris Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/7cf2da4bf549ed01c17f206327be4db7_valles_marineris_enhanced.tif_full.jpg'}]